<a href="https://colab.research.google.com/github/EbsHirani/FrenchTranslation/blob/master/Oui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.chdir("drive/My Drive/oui")


In [0]:
import pandas as pd
import numpy as np


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
fre_input = fre.apply(lambda x: "<BOS> " + x.lower()+ " <EOS>")
fre = fre.apply(lambda x: x.lower() + " <EOS>")
eng = eng.apply(lambda x: x.lower())

def tokenize(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(data)
  return tokenizer.texts_to_sequences(data), tokenizer
def pad_seq(sequences):
  padded = pad_sequences(sequences, padding = "post")
  return padded

In [0]:
eng_seq, eng_tok = tokenize(eng)
fre_input_seq, fre_tok = tokenize(fre_input)
fre_output_seq = fre_tok.texts_to_sequences(fre)
eng_seq = pad_seq(eng_seq)
fre_output_seq = pad_seq(fre_output_seq)
fre_input_seq = pad_seq(fre_input_seq)
fre_output_seq = fre_output_seq.reshape(fre_output_seq.shape[0], fre_output_seq.shape[1], 1) #for sparse categorical crsooentropy
eng_max = eng_seq.shape[1]
fre_max = fre_input_seq.shape[1]
eng_vocab_len = len(eng_tok.word_indices)
fre_vocab_len = len(fre_tok.word_indices)
fre_idx = {ids: word for word, ids in fre_tok.word_index.items()}
fre_wordx = {word: ids for word, ids in fre_tok.word_index.items()}
index_to_words[0] = '<PAD>' 


In [0]:
import pandas as pd 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten, LSTM, CuDNNLSTM
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.utils.np_utils import to_categorical


from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization , MaxPooling2D, Activation
LR = 1e-3
enc_input = Input(shape = (None,))
enc_emb = Embedding(eng_vocab_len, 64)(enc_input)
enc_lstm =  CuDNNLSTM(50, return_state = True)
_ , state_h, state_c = enc_lstm(emb)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None,))
dec_emb = Embedding(fre_vocab_len, 64)
dec_emb_out = dec_emb(dec_input)
dec_lstm = CuSNNLSTM(50, return_sequences = True, return_state = True)
dec_out, _, _ = dec_lstm(dec_emb_out, initial_state = enc_states)
dec_soft = Dense(fre_vocab_len, activation = "softmax")
dec_pred = dec_soft(dec_out)

model = Model([enc_input, dec_input], dec_pred)

In [0]:
model.compile(optimizer = Adam(LR), loss = sparse_categorical_crossentropy, metrics = ["accuracy"])


In [0]:
model.fit([eng_seq, fre_input_seq], fre_output, batch_size = 128, epochs = 100, callbacks =[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50), ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)])

In [0]:
enc_model = Model(enc_input, enc_states)
dec_state_in = [Input(shape = (50,)),Input(shape = (50,))]
dec_emb_out_pred = dec_emb(dec_inputs) 
dec_out_pred, state_h_out, state_c_out = dec_lstm(dec_emb_out_pred, initial_state = dec_state_in)
dec_out_pred = dec_soft(dec_out_pred)
dec_model = Model([dec_inputs, dec_state_in], [dec_out_pred, [state_h_out, state_c_out]])


In [0]:
def predict_french(seq):
  dec_init = enc_model.predict(seq)
  dec_word = np.array([[fre_wordx["<BOS>"]]])
  stop = False
  fre_sentence = ""
  while not stop:
      output = dec_model.predict([dec_seq, dec_init])
      output_seq = output[0]
      new_states = output[1]
      new_id = np.argmax(ouput_seq[0,-1,:])
      new_word = fre_id[new_id]
      fre_sentence += new_word + " "

      if (new_word = "<EOS>" or len(fre_sentence.split()) == fre_max):
        stop = True
      dec_word = np.array([[new_id]]) 
      dec_init = new_states
  return fre_sentence



